# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint as pp
from ipywidgets.embed import embed_minimal_html

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
sourceWeather = pd.read_csv('../output_data/weather_data.csv')
sourceWeather.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,MaxTemp,Temp,Windspeed
0,0,jalu,0,LY,1569648341,56,29.03,21.55,75.87,75.87,2.51
1,1,ushuaia,40,AR,1569648172,70,-54.81,-68.31,46.40,43.27,4.16
2,2,avarua,28,CK,1569648079,73,-21.21,-159.78,73.40,73.40,6.93
3,3,bowen,8,AU,1569648342,48,-20.01,148.25,84.20,84.20,10.29
4,4,albany,20,US,1569648232,71,42.65,-73.75,59.00,52.05,2.35


In [3]:
# Drop NaNs
sourceWeather = sourceWeather.dropna()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure GMaps
gmaps.configure(api_key=g_key)

In [5]:
# Store latitude and longitude into  locations 
locations = sourceWeather[["Latitude", "Longitude"]].astype(float)

# Convert Poverty Rate to float and store
humidity = sourceWeather["Humidity"].astype(float)

In [14]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig
embed_minimal_html('export.html', views=[fig])

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.
* Drop any rows will null values.

In [7]:
# Narrow down the weather df to just ideal cities
idealWeather = sourceWeather[sourceWeather.MaxTemp > 70]
idealWeather = idealWeather[sourceWeather.MaxTemp < 80]
idealWeather = idealWeather[sourceWeather.Windspeed < 10]
idealWeather = idealWeather[sourceWeather.Cloudiness == 0]
idealWeather.head()


C:\Users\mahlo\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\mahlo\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
C:\Users\mahlo\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,MaxTemp,Temp,Windspeed
0,0,jalu,0,LY,1569648341,56,29.03,21.55,75.87,75.87,2.51
49,49,port alfred,0,AU,1569648356,60,-31.65,116.67,71.60,71.60,8.05
54,54,tambo,0,GY,1569648194,100,6.80,-58.16,75.20,75.20,5.17
111,111,surskoye,0,GF,1569648223,94,4.94,-52.33,75.20,75.20,6.60
184,184,veraval,0,LY,1569648396,40,26.59,12.77,74.13,74.13,7.02


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = idealWeather

In [32]:
hotel_df = hotel_df.reset_index()

In [10]:
# Hard coded sample for transversal
target_coordinates = "43.6187102, -116.2146068"
target_search = "Hotel"
target_radius = 8000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params).json()
pp.pprint(response)

{'html_attributions': [],
 'next_page_token': 'CrQCIwEAALe9qy7g-PVTu_Rd1lWM8_8vq5POBeROGfedU3oTfOHEiTS3DNAf08wd9mpGAv55eMftpveP6nuz6yZifcX7_spo7a-QDhRoraTkYKgORc8yaFpUVQDs0OlNI2YvNsBmk9wZiTLQpUFnxf5-mOwlzABBosrJ-2u1X4VHUS0hhLhX9oATymb8_hF_y3gv1dCuNNWMrM1y2Xl2lNrmXDSY-niy_FbXd3SEtPh3-ONTjWObFecagPOptGHHQZ3-8ZCl1FKmqqzhOQXxdMbgo9pkoVGa-gk8Q-j1SnzKCLZIG8uy5jwUYjD3wtH7UKXP6yro2z5zZWThLp35c7r7M-ZP7w4k6oYBnw1C0xP40SP9yCCpKOaXNDIlFlWTPnYG4JUHYQYmtMxb86hKFOr_biB3gqYSEKaZYiK1XCCRQ47tgfwC8DEaFGe7quRXaENXgxNX70Y4cyMha-HX',
 'results': [{'geometry': {'location': {'lat': 43.6183199, 'lng': -116.2105384},
                           'viewport': {'northeast': {'lat': 43.61972197989272,
                                                      'lng': -116.2094804701073},
                                        'southwest': {'lat': 43.61702232010728,
                                                      'lng': -116.2121801298928}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lo

In [15]:
name = response['results'][0]['name']
name

'Modern Hotel'

In [16]:
hotelLatitude = response['results'][0]['geometry']['location']['lat']
hotelLatitude

43.6183199

In [17]:
hotelLongitude = response['results'][0]['geometry']['location']['lng']
hotelLongitude

-116.2105384

In [33]:
#Loop through each row to pull hotel data from Google
total = len(hotel_df)
rowCount = 1

hotel_df['Hotel'] = ""
hotel_df['Hotel Latitude'] = ""
hotel_df['Hotel Longitude'] = ""

for index, row in hotel_df.iterrows():
    print(f"Processing row #{index} of {total}")
    
    lat = str(row['Latitude'])
    lon = str(row['Longitude'])
    
    target_coordinates = lat + ", " + lon
    target_search = "Hotel"
    target_radius = 5000
    target_type = "hotel"
    
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Request the hotel data using the parameters
    response = requests.get(base_url, params=params).json()
    
    # Traverse and store the hotel details
    row['Hotel'] = response['results'][0]['name']
    row['Hotel Latitude'] = response['results'][0]['geometry']['location']['lat']
    row['Hotel Longitude'] = response['results'][0]['geometry']['location']['lng']
          
    rowCount += 1

Processing row #0 of 12
Processing row #1 of 12
Processing row #2 of 12
Processing row #3 of 12
Processing row #4 of 12


IndexError: list index out of range

In [34]:
hotel_df

,index,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,MaxTemp,Temp,Windspeed,Hotel,Hotel Latitude,Hotel Longitude
0,0,0,jalu,0,LY,1569648341,56,29.03,21.55,75.87,75.87,2.51,,,
1,49,49,port alfred,0,AU,1569648356,60,-31.65,116.67,71.60,71.60,8.05,,,
2,54,54,tambo,0,GY,1569648194,100,6.80,-58.16,75.20,75.20,5.17,,,
3,111,111,surskoye,0,GF,1569648223,94,4.94,-52.33,75.20,75.20,6.60,,,
4,184,184,veraval,0,LY,1569648396,40,26.59,12.77,74.13,74.13,7.02,,,
5,202,202,antofagasta,0,ES,1569648401,77,36.01,-5.60,71.01,64.53,3.36,,,
6,309,309,bilma,0,JP,1569648434,68,39.38,140.05,73.40,71.76,4.70,,,
7,336,336,mangrol,0,UZ,1569648442,17,38.20,67.20,74.78,74.78,7.49,,,
8,347,347,hvolsvollur,0,SY,1569648445,41,35.01,37.05,74.54,74.54,4.88,,,
9,424,424,bada,0,TR,1569648467,49,36.61,34.31,74.78,74.78,1.88,,,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map